### File 09: GLM Regression with Tweedie

Our response data is very 0-heavy. We createed a regression that accounts for this by using a Tweedie loss function, however, we were unable to get it to run with parameters that allowed it to create a mixture model. The error we got is 'sum of weights cannot be zero'. Our research suggests that this is an error in the pyspark implementation of the Tweedie/GLM model. It works for fake data, but the structure of ours causes it to break. 

### Set up Spark session

We can specify more options in the SparkSession creator, but currently the options are at the default settings.

In [1]:
%%time
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col
from pyspark.sql.functions import log
from pyspark.ml.stat import Correlation

import pandas as pd
import numpy as np
import copy

spark = SparkSession.builder \
        .appName("project") \
        .getOrCreate()

sc = spark.sparkContext

CPU times: user 614 ms, sys: 458 ms, total: 1.07 s
Wall time: 8.91 s


### Read in dataframes for train and test sets

This data should have been previously generated: we can find it in the `processed_data` folder.

In [2]:
%%time
trainDF = spark.read.parquet("./processed_data/train.parquet")
testDF = spark.read.parquet("./processed_data/test.parquet")


+---------+------------------+------------+--------------+-----------------+------------------+------------------+----------------------------+---------------------------+----------------------------+----------------------------+------------------------+------------------------+-------------------------+------------------------+-----------+---------------+-----------+----------------------+------------------+------------------+-------------------------+---------------------+------------------+------------------+--------------------+--------------------+----------------------+------------------+--------------------+--------------------+--------------------+--------------------+
|  user_id|       total_spend|total_events|total_sessions|    T_total_spend|avg_session_length| sd_session_length|avg_interactions_per_session|sd_interactions_per_session|max_interactions_per_session|purchase_pct_of_total_events|view_pct_of_total_events|cart_pct_of_total_events|avg_purchases_per_session|sd_purcha

In [3]:
# trainDF = trainDF \
#           .withColumn("total_spend_log", log(col("total_spend"))) \
#           .withColumn("total_events_log", log(col("total_events"))) \
#           .withColumn("purchase_events_log", log(col("purchase_events"))) \
#           .withColumn("total_sessions_log", log(col("total_sessions"))) \
#           .withColumn("T_total_spend_log", log(col("T_total_spend"))) \
#           .withColumn("total_spend_pos", (col("total_spend"))) \
#           .withColumn("total_events_pos", (col("total_events"))) \
#           .withColumn("purchase_events_pos", (col("purchase_events"))) \
#           .withColumn("total_sessions_pos", (col("total_sessions"))) \
#           .withColumn("T_total_spend_pos", (col("T_total_spend")/100+1))

# testDF = testDF \
#           .withColumn("total_spend_log", log(col("total_spend"))) \
#           .withColumn("total_events_log", log(col("total_events"))) \
#           .withColumn("purchase_events_log", log(col("purchase_events"))) \
#           .withColumn("total_sessions_log", log(col("total_sessions"))) \
#           .withColumn("T_total_spend_log", log(col("T_total_spend"))) \
#           .withColumn("total_spend_pos", (col("total_spend"))) \
#           .withColumn("total_events_pos", (col("total_events"))) \
#           .withColumn("purchase_events_pos", (col("purchase_events"))) \
#           .withColumn("total_sessions_pos", (col("total_sessions"))) \
#           .withColumn("T_total_spend_pos", (col("T_total_spend")/100+1))


### Set up Spark ML pipeline training for generalized linear regression

Here we decide which input columns should be used in order to create our training pipeline. To implement this step, we create the function `generatePipeline(inputCols, outputCol`). Then, we train the pipeline using this function.

In [4]:
%%time

inputCols = ["total_spend","total_events", "total_sessions", "avg_session_length", "avg_interactions_per_session", "max_interactions_per_session",
             "purchase_pct_of_total_events", "view_pct_of_total_events", "cart_pct_of_total_events","avg_purchases_per_session", "cart_events", "purchase_events",
             "view_events", "sessions_with_purchase", "sessions_with_cart","sessions_with_view", "pct_sessions_end_purchase", "pct_sessions_end_cart", 'sd_session_length', 
             'sd_interactions_per_session', 'sd_purchases_per_session', "total_spend_log", "total_events_log", "purchase_events_log", "total_sessions_log"]
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.GeneralizedLinearRegression.html
# on choosing variable values: https://www.rdocumentation.org/packages/statmod/versions/1.4.36/topics/tweedie
# LinkPower: Supported variables:
# variancePower: Supported values 0 and [1, inf)

def generateGLRPipeline(inputCols, outputCol):
    # Select input columns for generalized linear regression
    vecAssembler = VectorAssembler(inputCols=inputCols, outputCol="features")
    
    # create glr instance & select output col
    glr = GeneralizedLinearRegression(featuresCol = "features", labelCol = outputCol, family = "tweedie")

    pipeline = Pipeline(stages=[vecAssembler, glr])
    return pipeline
    
pipeline = generateGLRPipeline(inputCols, "T_total_spend")
pipelineGLRModel = pipeline.fit(trainDF)

CPU times: user 13.5 ms, sys: 9.28 ms, total: 22.8 ms
Wall time: 5.05 s


In [5]:
def modelInfo(inputCols, pipelineGLRModel):
    # Create a zipped list containing the coefficients and the data
    modelCols = copy.deepcopy(inputCols)
    modelCoeffs = list(pipelineGLRModel.stages[-1].coefficients)
    modelCoeffs.insert(0,pipelineGLRModel.stages[-1].intercept)
    modelCols.insert(0,"intercept")
    modelZippedList = list(map(list, zip(modelCols, modelCoeffs)))

    # Create the pandas DataFrame
    modelDF = pd.DataFrame(modelZippedList, columns = ['Column name', 'Coefficient'])
    return modelDF

print("Model coefficients")
print(modelInfo(inputCols, pipelineGLRModel))


Model coefficients
                     Column name  Coefficient
0                      intercept   476.265157
1                    total_spend     0.441111
2                   total_events     0.032116
3                 total_sessions    74.283336
4             avg_session_length    -0.000279
5   avg_interactions_per_session    -1.511373
6   max_interactions_per_session     1.584360
7   purchase_pct_of_total_events  -138.048509
8       view_pct_of_total_events   119.843074
9       cart_pct_of_total_events  -202.344819
10     avg_purchases_per_session  -288.082209
11                   cart_events     8.267930
12               purchase_events    38.253511
13                   view_events    -0.286638
14        sessions_with_purchase   -49.390201
15            sessions_with_cart    45.325245
16            sessions_with_view   -75.709854
17     pct_sessions_end_purchase    79.035341
18         pct_sessions_end_cart  -227.206520
19             sd_session_length     0.000458
20   sd_interac

In [6]:
# Calculate adjusted r2 (https://towardsdatascience.com/machine-learning-linear-regression-using-pyspark-9d5d5c772b42)
def adj_r2(r2, inputCols, testDF):
    n = testDF.count()
    p = len(inputCols)
    adjusted_r2 = 1-(((1-r2)*(n-1))/(n-p-1))
    return adjusted_r2

In [7]:
def getEvaluationMetrics(pipelineModel,outputCol,testDF,inputCols):
    predDF = pipelineModel.transform(testDF)
    predDF.select(outputCol, "prediction").show(10)

    regressionEvaluator = RegressionEvaluator(
    predictionCol="prediction",
    labelCol=outputCol,
    metricName="rmse")
    rmse = regressionEvaluator.evaluate(predDF)

    regressionEvaluator = RegressionEvaluator(
    predictionCol="prediction",
    labelCol=outputCol,
    metricName="r2")
    r2 = regressionEvaluator.evaluate(predDF)
    
    # Manually calculate Adjusted r2
    adjusted_r2 = adj_r2(r2, inputCols, testDF)
    
    return rmse, r2, adjusted_r2

evaluationMetrics = getEvaluationMetrics(pipelineGLRModel,"T_total_spend",testDF,inputCols)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

+------------------+------------------+
|     T_total_spend|        prediction|
+------------------+------------------+
|               0.0| 101.8401348619305|
|               0.0|120.02613071545204|
|               0.0| 81.77208516023427|
|               0.0|164.42160417183175|
|               0.0| 72.81669088335002|
|               0.0| 52.22695699452589|
| 312.4200134277344|-46.68556425951215|
|  6009.78010559082|  578.043938617308|
|1627.1900024414062|199.25451415727548|
|1393.8800354003906|140.94823349995113|
+------------------+------------------+
only showing top 10 rows

RMSE is 2192.1
R^2 is 0.20860
Adjusted R^2 is 0.20832


#### While we want to do a tweedie model to test how a model built for zero-inflation works, it appears that this implementation is unable to create a mixture model using data such as ours. It generates an error 'Sum of weights cannot be zero' that we were unable to overcome - what we have found online indicates that this is an issue specifically with the pyspark implementation of this model. The model below uses data that we have coerced to the point that it is able to run, though the response variables are so modified as to be useless - we divided by 100, then added 1. Even then, it would not run unless we filtered to those with values originally <= 100. We provide it to prove only that the code works, tuning this model is useless, because it only works if all of the response values have had most of their variability scrubbed out. 

In [ ]:
#trainDF2 = trainDF.filter(col('T_total_spend') <=100)

##### Cross-evaluate tweedie variables (linkPower & varPower)

In [9]:
# For crossval to work, must define pipeline here
# Select input columns for generalized linear regression
vecAssembler = VectorAssembler(inputCols=inputCols, outputCol="features")

# create glr instance & select output col
glr = GeneralizedLinearRegression(featuresCol = "features", labelCol = "T_total_spend", family = "tweedie")

pipeline = Pipeline(stages=[vecAssembler, glr])

paramGrid = ParamGridBuilder() \
    .addGrid(glr.variancePower, [1, 2]) \
    .addGrid(glr.linkPower, [2, 1.5]) \
    .build()

crossval = CrossValidator(estimator = pipeline,
                         estimatorParamMaps=paramGrid,
                         evaluator=RegressionEvaluator().setLabelCol("T_total_spend"),
                         numFolds=4)
# Run cross-validation, and choose best set of parameters.
cvModel = crossval.fit(trainDF)

Py4JJavaError: An error occurred while calling o319.fit.
: java.lang.AssertionError: assertion failed: Sum of weights cannot be zero.
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.ml.optim.WeightedLeastSquares$Aggregator.validate(WeightedLeastSquares.scala:426)
	at org.apache.spark.ml.optim.WeightedLeastSquares.fit(WeightedLeastSquares.scala:108)
	at org.apache.spark.ml.optim.IterativelyReweightedLeastSquares.fit(IterativelyReweightedLeastSquares.scala:91)
	at org.apache.spark.ml.regression.GeneralizedLinearRegression.$anonfun$train$1(GeneralizedLinearRegression.scala:431)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.regression.GeneralizedLinearRegression.train(GeneralizedLinearRegression.scala:379)
	at org.apache.spark.ml.regression.GeneralizedLinearRegression.train(G

In [17]:
print("Best model coefficients")
print(modelInfo(inputCols, cvModel.bestModel))

evaluationMetrics = getEvaluationMetrics(cvModel.bestModel,"T_total_spend_pos",testDF,inputCols)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

print()
print(cvModel.getEstimatorParamMaps()[np.argmax(cvModel.avgMetrics)])

Best model coefficients
                     Column name   Coefficient
0                      intercept  1.008924e+00
1                    total_spend  1.068017e-06
2                   total_events  4.217552e-03
3                 total_sessions  2.033541e-03
4             avg_session_length -1.258126e-07
5   avg_interactions_per_session  3.255075e-05
6   max_interactions_per_session -5.862676e-06
7   purchase_pct_of_total_events  2.051163e-02
8       view_pct_of_total_events -1.410037e-03
9       cart_pct_of_total_events -7.652089e-03
10     avg_purchases_per_session -2.249452e-03
11                   cart_events -4.076877e-03
12               purchase_events -5.219651e-03
13                   view_events -4.207335e-03
14        sessions_with_purchase  1.335609e-03
15            sessions_with_cart -6.353159e-04
16            sessions_with_view -2.012615e-03
17     pct_sessions_end_purchase  4.937284e-03
18         pct_sessions_end_cart  3.780298e-03
19             sd_session_length  2.

In [ ]:
# Troubleshooting: https://stackoverflow.com/questions/49567921/pyspark-how-to-fit-a-glm-using-log-as-link-function-with-sum-of-weights-as-zero
# Sum of weights cannot be 0 issue
# Other suggestion is data is too long-tailed, try cutting it (treating more of spend as outlier)
#trainDF = trainDF.withColumn("T_total_spend_small", col("T_total_spend")/1000) # Per one person's advice, try making the response values smaller 

# code that produces "Sum of weights cannot be zero" is here: https://github.com/apache/spark/blob/master/mllib/src/main/scala/org/apache/spark/ml/optim/WeightedLeastSquares.scala
